In [10]:
import requests
import json
import pandas as pd
import psycopg2



In [ ]:
# ETL - Extract, Transform and Load

# Extract : Extraindo uma informação do seu local de Origem

# Faz uma requisição GET para a API do users do jsonplaceholder
res = requests.get("https://jsonplaceholder.typicode.com/users")
data = res.json()
data_df = pd.DataFrame(data)

# Transform : Transformando a informação para o formato desejado ( remove colunas desnecessárias )
data_df = data_df[["id", "name", "username", "email", "phone", "website"]]
data_df.head()

,id,name,username,email,phone,website
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,010-692-6593 x09125,anastasia.net
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,1-463-123-4447,ramiro.info
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,493-170-9623 x156,kale.biz
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,(254)954-1289,demarco.info


In [18]:

host = 'localhost'
database = 'mydatabase'
user = 'myuser'
password = 'mypassword'
port = '5432'

conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)
cur = conn.cursor()


In [12]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY,
        name TEXT,
        username TEXT,
        email TEXT,
        phone TEXT,
        website TEXT
    )
""")

conn.commit()

In [ ]:
# Load : Carregando a informação transformada para o local de destino ( banco de dados )

for index, row in data_df.iterrows():
    cur.execute("""
        INSERT INTO users (id, name, username, email, phone, website)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (id) DO NOTHING
    """, (row['id'], row['name'], row['username'], row['email'], row['phone'], row['website']))
    
    conn.commit()

In [19]:
results = pd.read_sql("SELECT * FROM users", conn)
print(results)

   id                      name          username                      email  \
0   1             Leanne Graham              Bret          Sincere@april.biz   
1   2              Ervin Howell         Antonette          Shanna@melissa.tv   
2   3          Clementine Bauch          Samantha         Nathan@yesenia.net   
3   4          Patricia Lebsack          Karianne  Julianne.OConner@kory.org   
4   5          Chelsey Dietrich            Kamren   Lucio_Hettinger@annie.ca   
5   6      Mrs. Dennis Schulist  Leopoldo_Corkery    Karley_Dach@jasper.info   
6   7           Kurtis Weissnat      Elwyn.Skiles     Telly.Hoeger@billy.biz   
7   8  Nicholas Runolfsdottir V     Maxime_Nienow       Sherwood@rosamond.me   
8   9           Glenna Reichert          Delphine    Chaim_McDermott@dana.io   
9  10        Clementina DuBuque    Moriah.Stanton     Rey.Padberg@karina.biz   

                   phone        website  
0  1-770-736-8031 x56442  hildegard.org  
1    010-692-6593 x09125  anastasia

C:\Users\James\AppData\Local\Temp\ipykernel_23424\3150360745.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results = pd.read_sql("SELECT * FROM users", conn)


In [20]:
cur.close()
conn.close()